## Trenowanie modelu liniowego w tensorflow

### Zadanie 1 - wczytanie danych

W ramach zadania podejmiesz wyzwanie kaggle kc_house_data.
https://www.kaggle.com/datasets/shivachandel/kc-house-data

Udostępnione w ramacha wyzwania dane zawierają ceny domów sprzedanych pomiędzy majem 2014, a majem 2015 w hrabstwie King County stanu Waszyngton w USA. Dane wczytaj za pomocą biblioteki pandas.

In [ ]:
# zaimportuj bibliotekę pandas i nadaj jej alias pd
___

# przypisz url pliku csv do zmiennej data_url
___ = "https://raw.githubusercontent.com/jgrynczewski/tensorflow_intro/master/kc_house_data.csv"

# wczytaj plik za pomocą funkcji read_csv, przekazując do niej url pliku. 
# Wynik przypisz do zmiennej data
___ = pd.read_csv(___)

# wyświetl dataframe z wczytanymi danymi
data

Zbiór zawiera informacje o ponad 21000 domów. Każdy dom opisany jest 21 cechami, wsród których znajduje się cena sprzedaży (kolumna *price*). Dane w niektórych kolumnach są typu numerycznego - *floors*, *price*, *bedrooms*, inne typu boolean - *waterfront*. Kolumna *date* zawiera datę zawarcia transakcji.

In [ ]:
data.columns

### Zadanie 2 - rzutowanie danych

Zrzutuj dane z kolumn price na tablicę numpy, a dane z kolumny waterfront na obiekt typu tf.Tensor. Pamiętaj o nadaniu obiektom odpowiednich typów (tablica numpy - float32, tf.tensor - tf.bool).

In [ ]:
# Zaimportuj biblioteki numpy i tensorflow z ich standardowymi aliasami
___
___

# Zrzutuj dane z kolumny price na tablicę numpy typu np.float32
price = np.array(data['price'], np.___)

# Zrzutuj dane z kolumny waterfront na obiekt tf.Tensor typu tf.bool
# Do rzutowania użyj funkcji tf.cast
waterfront = tf.___(data['waterfront'], tf.___)

# Wyświetl zmienną price
print(___)

# Wyświetl zmienną waterfront
print(___)

### Zadanie 3 - estymacja błędu (funkcja kosztu)

Ceny domów przechowujesz w tablicy numpy *price* (poprzednie zadanie). Przy założeniu, że przewidywane, obliczone przez model ceny przechowywane są w tablicy numpy *predictions* znajdź średni błąd kwadratowy predykcji (mse). Dane predictions są załadowane z pliku csv poniżej.

In [ ]:
# Załadowanie predykcji do zmiennej predicions
# ceny domów są już przechowywane w zmiennej price (zadanie 2)

predictions_url = "https://raw.githubusercontent.com/jgrynczewski/tensorflow_intro/master/predictions.csv"

predictions_data = pd.read_csv(predictions_url, header=None)

predictions = predictions_data.to_numpy().flatten()

In [ ]:
# policz średni błąd kwadratowy predykcji (mean squared error - mse)
loss = tf.keras.losses.___(price, predictions)

# wyświetl wartość błędu
print(loss.numpy())

In [ ]:
# policz średni błąd absolutny predykcji (mean absoulte error - mae)
loss = tf.keras.losses.___(price, predictions)

# wyświetl wartość błędu
print(loss.numpy())

Jak pewnie zauważyłeś mae ma znacznie mniejszą wartość niż mse mimo, że wartości predictions i price były takie same w obu przypadkach. Dzieje się tak dlatego, że różne funkcje kosztów różnie każą za niezgodność prognozy z ceną rzeczywistą. MSE szczególnie nie lubi dużych niezgodności i mocno każe za takie niezgodności.

### Zadanie 4 - predykcja

W poprzednim zadaniu wczytałeś dane predykcji z pliku. W tym zadaniu napiszesz własny model, który sam będzie prognozował wynik. Najprostszy, liniowy model na postać:

$$ 
predictions = scalar * features
$$

, gdzie *scalar* to parametr (waga), którego wartość należy tak dobrać, aby funkcja kosztu miała wartość najmniejszą, tzn. taką wartość która w najdokładniejszy możliwy sposób przeniesie wektor cech na wektor celu.

Dla uproszczenia obliczeń wektor cech ma postać

$$
features = [1, 2, 3, 4, 5]
$$

a wektor celu (w poprzednim zadaniu to były ceny) ma postać

$$
targets = [2, 4, 6, 8, 10]
$$ 



In [ ]:
features = tf.constant([1, 2, 3, 4, 5], dtype=tf.float32)
targets = tf.constant([2, 4, 6, 8, 10], dtype=tf.float32)

Zapewne od razu widzisz poszukiwaną wartośc prametru *scalar*. Wartość 2 to wartość, dla której wektor *predictions* będzie nie tylko podobny, ale nawet identyczny jak wektor targets, a wartość funkcji kosztów wyniesie 0. Ale zrób to zadanie krok po kroku:
1. zdefiniuj model
2. zdefiniuj własną funkcję loss_function, która będzie liczyła prognozę modelu i zwracała jej błąd

In [ ]:
# Wartość parametru skalar (zawsze trzeba przyjąc jakąś wartość początkową)
# Zainicalizuj tensor tf.Variable typu float32 o wartości 1 i przypisz go do 
# zmiennej scalar (to jest pierwsza wartość parametru scalar)
scalar = ___(1.0, tf.___)

# zdefiniuj model
def model(scalar, features = features):
  	return scalar * features

# Zdefiniuj funkcję loss_function
def loss_function(___, features = features, targets = targets):
	"""Liczy predykcję oraz jej błąd. Zwraca błąd predykcji"""

  # policz predykcje dla zadanej wartości parametru scalar
	predictions = model(scalar, features)
    
	# policz i zwróć średni błąd absolutny predykcji
	return tf.keras.losses.___(targets, predictions)

# Wykonaj funkcje compute_loss i wyświetl koszt (błąd predykcji) (dla wartość 
# parametry scalar=1.0 - przypominam, bo wartość parametru scalar zdefiniowałeś 
# na samym początku)
print(loss_function(scalar).numpy())

## Zadanie 5 - jednowymiarowa regresja liniowa

W tej chwili dysponujesz już wszystkim co jest potrzebne do zaimplementowania regresji liniowej. Dodając człon stały (intercept) do liniowego modelu z poprzedniego zadania otrzymasz pełny wzór regresji liniowej.

$$ 
prediction = intercept + slope * features
$$

W tym przypadku mówimy o jednowymiarowej regresji liniowej. Jednowymiarowa regresja liniowa to model identyfikujący zależność pomiędzy pojedynczą cechą a wektorem celu. W ramach zadania należy odgadnąć cenę domu na podstawie jego powierzchni. W tym celu zdefniujesz model oraz funkcję loss_function i policzysz jej wartość dla dwóch różnych wartości intercept i slope.

**Instrukcja**
* zdefinuj funkcję, która zwróci przewidywaną wartość dla regresji liniowej przy użyciu *intercept*, *features* i *slope*
* uzupełnij loss_function() poprzez dodanie parametrów intercept i slope
* policz mse na podstawie targets i predictions.

In [ ]:
# Najpierw popatrz na dane
import matplotlib.pyplot as plt

print(data.columns)

size = np.array(data['sqft_lot'], np.float32)

plt.plot(size, price, 'o', color='black')
plt.xlabel('size')
plt.ylabel('price')
plt.show()

In [ ]:
# Mocno rozrzucone. A log ?

size_log = np.log(size)
price_log = np.log(price)
print(size_log)
plt.plot(size_log, price_log, 'o', color='black')
plt.xlabel('log(size)')
plt.ylabel('log(price)')
plt.show()

In [ ]:
# Lepiej. Dlaczego log ?

data.hist('price',figsize=(8,5))
plt.title('Number of houses vs Price')
plt.ylabel('Number of Houses')
plt.xlabel("Price")

In [ ]:
data['log_price'] = np.log(data['price'])

data.hist('log_price',figsize=(8,5))
plt.title('Number of houses vs log(Price)')
plt.ylabel('Number of Houses')
plt.xlabel("log(Price)")  # Tłumi różnice w dużych wartościach (patrz wykres log(x))

In [ ]:
# Zdefiniuj model regresji liniowej (y = b + a*x)
def linear_regression(intercept, slope, features = size_log):
	return ___

# Zdefiniuj loss_function() w taki sposób, żeby przyjmowała odpowiednie zmienne
# jako argumenty
def loss_function(___, ___, features = size_log, targets = price_log):
	# Predykcja
	predictions = linear_regression(intercept, slope, features)
    
    # Zwróc wartośc mse
	return tf.keras.losses.___

# Oblicz wartość kosztu dla zadanych wartości parametrów slope i intercept
print(loss_function(0.1, 0.1).numpy())
print(loss_function(0.1, 0.5).numpy())

### Zadanie 6 - trenowanie modelu liniowego (do przeczytania)


W jaki sposób zbiegać z wartościami parametrów intercept i slope ?
Wzdłuż największego spadku funkcji kosztów (wzdłuż spadku gradientu). Z jakim krokiem ?
Nie będziemy tego wszystkiego samodzielnie implementować. W tym miejscu używa się tzw. optymalizatorów. Użyjemy optymalizatora Adam.

In [ ]:
# Początkowe wartości parametrów intercept, slope (przyjęte arbitralnie)
intercept = tf.Variable(5, dtype=tf.float32)
slope = tf.Variable(0.001, dtype=tf.float32)

# Inicjalizacja opytmalizatora
opt = tf.keras.optimizers.Adam(0.5)

for j in range(100):
	# Zastosuj metodę minimize optymalizatora. Do metody przekaż funkcję kosztów
	# i wartości parametrów
	opt.minimize(lambda: loss_function(intercept, slope), var_list=[intercept, slope])

	# Co dziesiąte przejście wyświetlaj koszt
	if j % 10 == 0:
		print(loss_function(intercept, slope).numpy())

# Wyświetl otrzymane wartości parametrów
print(intercept)
print(slope)

# Wyświetl dane oraz model dla otrzymanych wartości parametrów intercept i slope
y_predict = intercept + slope * size_log

# dane
plt.plot(size_log, price_log, 'o', color='black', markersize=2)

# model
plt.plot(size_log, y_predict, color='red')

plt.grid()
plt.xlabel('log(size)')
plt.ylabel('log(price)')
plt.show()

### Zadanie 7 - wielowymiarowa regresja liniowa

Jedna cecha (w poprzednim zadaniu była to powierzchnia domu) zazwyczaj to za mało informacji, żeby móc trafnie prognozować wynik. Najczęściej wynik zależy od kilku, często wielu czynników. Na cenę domów napewno poza powierzchnią wpływ mają takie czynniki jak liczba sypialni czy bliskość wybrzeża. W celu uwzględnienia wielu czynników należy użyć regresji wielowymiarowej. W swojej istocie regresja wielowymiarowa nie różni się niczym od regresji jednowymiarowej. Zagadnienie zwiększa jedynie swoją wymiarowość czego efektem jest macierz (a nie wektor) cech, oraz wektor z wartościami jednego intersect i po jednym slope dla każdej z cech (a nie pojedyncza wartość slope).

W zadaniu uwzględnij dwie cechy - powierzchnię oraz liczbę sypialni (regresja dwuwymiarowa).

W przypadku regresji dwuwymiarowej równanie modelu ma postać:
$$
predictions = intersect + slope_{1} \cdot feature_1 + slope_{2} \cdot feature_{2} 
$$

In [ ]:
bedrooms = np.array(data['bedrooms'], np.float32)

In [ ]:
# Wartości początkowe parametrów (przyjęte doświadczalnie, takie przy których
# model stosunkowo szybko zbiega do poprawnego wyniku)
params = tf.Variable(np.array([10, 0.65, 0.2]))

# Zdefiniuj dwuwymiary model regresji liniowej
def linear_regression(params, feature1 = size_log, feature2 = bedrooms):
	return params[0] + feature1*___ + feature2*___

# Zdefiniuj funkcje kosztów
def loss_function(___, targets = price_log, feature1 = size_log, feature2 = bedrooms):
	# Predykcja
	predictions = linear_regression(params, feature1, feature2)
  
	# Średni błąd absolutny
	return tf.keras.losses.___(targets, predictions)

# Optymizator Adam
opt = tf.keras.optimizers.Adam()

# Minimalizacja funkcji kosztów (za pomocą optymalizatora)
for j in range(1000):
  opt.minimize(lambda: loss_function(___), var_list=[___])
  
  # Wyświetlanie parametrów oraz wartości funkcji kosztów dla nich w co 
  # 10-tym kroku
  if j%10 == 0:
    loss = loss_function(params)
    print(loss.numpy())
    print(params)

In [ ]:
# Wyświetl dane i model

# Wykres 3d
fig = plt.figure()
ax = plt.axes(projection='3d')

x_dots = size_log
y_dots = bedrooms
z_dots = price_log

# zbiór punktów
ax.scatter3D(x_dots, y_dots, z_dots, color='black');

x_min = np.min(x_dots)
x_max = np.max(x_dots)

y_min = np.min(y_dots)
y_max = np.max(y_dots)

x = np.linspace(x_min, x_max, 100)
y = np.linspace(y_min, y_max, 100)
z = params[0] + params[1] * x + params[2] * y

# model
ax.plot3D(x, y, z, color='red')

# etykiety osi
ax.set_xlabel('log(size)')
ax.set_ylabel('bedrooms')
ax.set_zlabel('log(price)')

plt.show()